In [ ]:
# ===== process GPS VTEC by minute & hour (all sheets) =====
import numpy as np
import pandas as pd
from google.colab import files

# 1) Upload the Excel file (shows a "Choose File" button)
uploaded = files.upload()
if not uploaded:
    raise SystemExit("No file uploaded.")
in_name = list(uploaded.keys())[0]

# 2) Read the workbook
xls = pd.ExcelFile(in_name)

# 3) Prepare writer for the output workbook
out_name = in_name.rsplit(".", 1)[0] + "_processed.xlsx"
writer = pd.ExcelWriter(out_name, engine="xlsxwriter")

for sheet in xls.sheet_names:
    # --- Load one sheet ---
    df = pd.read_excel(xls, sheet_name=sheet)
    if df.shape[1] < 4:
        # Ensure we at least have the first 4 columns
        raise ValueError(f"Sheet '{sheet}' must have at least 4 columns: Week_GPS, SOW, Satellite_ID, Vtec")

    # Keep only first 4 columns and standardize names (your SOW stays unchanged)
    df = df.iloc[:, :4].copy()
    df.columns = ["Week_GPS", "SOW", "Satellite_ID", "Vtec"]

    # Types (do not modify the SOW column itself)
    df["Satellite_ID"] = pd.to_numeric(df["Satellite_ID"], errors="coerce").astype("Int64")
    df["Vtec"] = pd.to_numeric(df["Vtec"], errors="coerce")
    df["SOW_num"] = pd.to_numeric(df["SOW"], errors="coerce")  # helper only for sorting

    # GPS only (1..37)
    df = df[(df["Satellite_ID"] >= 1) & (df["Satellite_ID"] <= 37)]

    # If nothing left, still write the empty original block and skip summaries
    if df.empty:
        df[["Week_GPS","SOW","Satellite_ID","Vtec"]].to_excel(
            writer, sheet_name=sheet, startrow=0, startcol=0, index=False
        )
        continue

    # --- Minute Index: 1,2,3... from sorted unique SOW values (no repeats) ---
    sows_sorted = np.sort(df["SOW_num"].dropna().unique())
    minute_map = {val: idx + 1 for idx, val in enumerate(sows_sorted)}
    df["Minute_Index"] = df["SOW_num"].map(minute_map)

    # --- Minute summary (skip NaNs in Vtec automatically) ---
    minute_summary = (
        df.dropna(subset=["Minute_Index"])
          .groupby("Minute_Index", sort=True)["Vtec"]
          .mean()
          .reset_index(name="Minute_Avg_Vtec")
    )

    # --- Hour index from minute index (1..24) and hour summary ---
    minute_summary["Hour_Index"] = ((minute_summary["Minute_Index"] - 1) // 60) + 1
    hour_summary = (
        minute_summary.groupby("Hour_Index")["Minute_Avg_Vtec"]
                      .mean()                     # mean over minutes in the hour
                      .reindex(range(1, 25))     # force 1..24 hours
                      .reset_index()
                      .rename(columns={"Minute_Avg_Vtec": "Hour_Avg_Vtec"})
    )

    # --- Write to the same sheet ---
    # Original data (A-D)
    df_out = df[["Week_GPS", "SOW", "Satellite_ID", "Vtec"]].copy()
    df_out.to_excel(writer, sheet_name=sheet, startrow=0, startcol=0, index=False)

    # Minute summary (G-H)
    minute_summary[["Minute_Index", "Minute_Avg_Vtec"]].to_excel(
        writer, sheet_name=sheet, startrow=0, startcol=6, index=False
    )

    # Hour summary (J-K)
    hour_summary[["Hour_Index", "Hour_Avg_Vtec"]].to_excel(
        writer, sheet_name=sheet, startrow=0, startcol=9, index=False
    )

    # Optional formatting
    ws = writer.sheets[sheet]
    ws.set_column(0, 3, 12)   # A-D widths
    ws.set_column(6, 7, 16)   # G-H widths
    ws.set_column(9,10, 14)   # J-K widths

# 4) Save and download
writer.close()
files.download(out_name)

